In [1]:
import pandas as pd
import glob

In [2]:
len(glob.glob('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/UW_HPRC_HiFi_Y4_AND_Y3_Topoff/3_data_processing/batch1-hifi_qc/input_jsons/*.json'))

119

In [3]:
topoff_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/Pangenome_HiFi_Libraries_Year4 - Topoff_Sequencing.tsv',sep='\t')
topoff_df

,Sample,Topoff Coverage (X),Total Coverage (X)
0,HG01891,21.2,64.2
1,HG02486,16.7,62.7
2,HG02559,15.5,63.5
3,HG02257,24.7,65.7
4,HG01358,NaN,43.0
...,...,...,...
145,HG03742,NaN,49.0
146,NA19043,12.1,60.5
147,NaN,NaN,NaN
148,NaN,NaN,NaN


In [4]:
def hprc_qc_review(output_df):
	pangenome_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/Pangenome_HiFi_Libraries_Year4 - Totals_by_Sample.csv')
	topoff_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/Pangenome_HiFi_Libraries_Year4 - Topoff_Sequencing.tsv',sep='\t')
	output_dict = {sample:[] for sample in output_df['aggregate-sample_id'].tolist()}
	for data in output_df[['aggregate-sample_id','hifi_qc_wf.hifi_qc_summary']].values:
		output_dict[data[0]].append(pd.read_csv(data[1],sep='\t'))

	coverage_dict = {}
	for k,v in output_dict.items():
		
		output_dict[k] = pd.concat(v)
		output_dict[k]['theoretical_coverage'] = [data/3.1 for data in output_dict[k]['total_Gbp'].tolist()]
		theoretical_coverage = output_dict[k]['theoretical_coverage'].sum()
		
		# +/- 1
		if pangenome_df[pangenome_df['Sample ID'].isin([k])]['Coverage'].tolist() != []:
			if round(theoretical_coverage,1) != pangenome_df[pangenome_df['Sample ID'].isin([k])]['Coverage'].tolist()[0]:
				print(k, round(theoretical_coverage),  round(pangenome_df[pangenome_df['Sample ID'].isin([k])]['Coverage'].tolist()[0]))
		# else:
		# 	print(k, round(theoretical_coverage,1))
		
		# # Topoff Coverage (X) not Total Coverage (X)
		# if topoff_df[topoff_df['Sample'].isin([k])]['Topoff Coverage (X)'].tolist() != []:
		# 	if round(theoretical_coverage,1) != topoff_df[topoff_df['Sample'].isin([k])]['Topoff Coverage (X)'].tolist()[0]:
		# 		print(k, round(theoretical_coverage),  topoff_df[topoff_df['Sample'].isin([k])]['Topoff Coverage (X)'].tolist()[0])
		# # else:
		# 	# print(k, round(theoretical_coverage,1))

	
		# if theoretical_coverage < 60:
		# 	coverage_dict[k] = theoretical_coverage
			# assert round(theoretical_coverage,1) == pangenome_df[pangenome_df['Sample ID'].isin([k])]['Coverage'].tolist()[0]
			# print(k, theoretical_coverage)
	return output_dict, coverage_dict


In [5]:
batch1_output_df = pd.read_csv('batch1-hifi_qc/hifi_samples_output_table.csv')
print(batch1_output_df.shape)
batch1_output_df['aggregate-sample_id'] = [sample.split('_')[0] for sample in batch1_output_df['sample_id'].tolist()]
# print(batch1_output_df[batch1_output_df['aggregate-sample_id'].isin(['HG03778'])])
batch1_output_dict, batch1_coverage_dict = hprc_qc_review(batch1_output_df)

(119, 11)
HG00126 64 64
HG00133 66 66
HG00272 68 68
HG00290 63 63
HG01028 66 66
HG03778 56 61
HG04225 67 67
NA19776 62 62


HG03778: 56 61 missing one file m84046_230731_200859_s2.hifi_reads.bc2095.bam
HG01361: 38 13.5
HG01123: 26 11.8


In [6]:
batch2_output_df = pd.read_csv('batch2-hifi_qc/hifi_samples_output_table.csv')
batch2_output_df['aggregate-sample_id'] = [sample.split('_')[0] for sample in batch2_output_df['sample_id'].tolist()]
batch2_output_dict, batch2_coverage_dict = hprc_qc_review(batch2_output_df)

HG01361 38 14
HG03017 17 17


In [7]:
batch2_coverage_df = pd.DataFrame.from_dict(batch2_coverage_dict, orient='index')
batch2_coverage_df.shape # topoff in addition to deepconsensus (sequel). check y1-y3 (no y4). 

(0, 0)

In [8]:
batch3_output_df = pd.read_csv('batch3-hifi_qc/hifi_samples_output_table.csv')
batch3_output_df['aggregate-sample_id'] = [sample.split('_')[0] for sample in batch3_output_df['sample_id'].tolist()]
batch3_output_dict, batch3_coverage_dict = hprc_qc_review(batch3_output_df)

batch3_coverage_df = pd.DataFrame.from_dict(batch3_coverage_dict, orient='index')
batch3_coverage_df # topoff in addition to deepconsensus (sequel). check y1-y3 (no y4). 

HG01123 26 12


""


In [9]:
batch4_output_df = pd.read_csv('batch4-hifi_qc/hifi_samples_output_table.csv')
# print(batch4_output_df.shape)
# print(batch4_output_df)
batch4_output_df['aggregate-sample_id'] = [sample.split('_')[0] for sample in batch4_output_df['sample_id'].tolist()]
# batch4_output_df
batch4_output_dict, batch4_coverage_dict = hprc_qc_review(batch4_output_df)
batch4_coverage_df = pd.DataFrame.from_dict(batch4_coverage_dict, orient='index')
batch4_coverage_df # topoff in addition to deepconsensus (sequel). check y1-y3 (no y4). 

""


In [10]:
output_df = pd.concat([batch1_output_df, batch2_output_df,batch3_output_df, batch4_output_df])
# output_df[output_df['aggregate-sample_id'].isin(['HG02486'])]
output_df[output_df['aggregate-sample_id'].isin(['HG03778'])]

,sample_id,hifi_reads,other_reads,perform_methylation_check,hifi_qc_wf.methylation_report,hifi_qc_wf.hifi_qc_summary,hifi_qc_wf.ntsm_eval,hifi_qc_wf.ext_ntsm_counts,hifi_qc_wf.hifi_ntsm_counts,hifi_qc_wf.hifi_readstat_report,hifi_qc_wf.hifi_readstat_tar,aggregate-sample_id
63,HG03778_m84046_230727_225904_s1.hifi_reads.bc2...,s3://human-pangenomics/submissions/4f8c8e63-df...,['/private/groups/hprc/human-pangenomics/1000G...,True,['/private/groups/hprc/human-pangenomics/HPRC_...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,HG03778
64,HG03778_m84046_230728_195023_s2.hifi_reads.bc2...,s3://human-pangenomics/submissions/4f8c8e63-df...,['/private/groups/hprc/human-pangenomics/1000G...,True,['/private/groups/hprc/human-pangenomics/HPRC_...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,HG03778
65,HG03778_m84046_230728_202047_s3.hifi_reads.bc2...,s3://human-pangenomics/submissions/4f8c8e63-df...,['/private/groups/hprc/human-pangenomics/1000G...,True,['/private/groups/hprc/human-pangenomics/HPRC_...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,HG03778
66,HG03778_m84046_230728_205153_s4.hifi_reads.bc2...,s3://human-pangenomics/submissions/4f8c8e63-df...,['/private/groups/hprc/human-pangenomics/1000G...,True,['/private/groups/hprc/human-pangenomics/HPRC_...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,HG03778
67,HG03778_m84046_230731_200859_s2.hifi_reads.bc2...,s3://human-pangenomics/submissions/4f8c8e63-df...,['/private/groups/hprc/human-pangenomics/1000G...,True,['/private/groups/hprc/human-pangenomics/HPRC_...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,HG03778


In [18]:
output_dict = {sample:[] for sample in output_df['aggregate-sample_id'].tolist()}
for data in output_df[['aggregate-sample_id','hifi_qc_wf.hifi_qc_summary']].values:
	output_dict[data[0]].append(pd.read_csv(data[1],sep='\t'))

for k,v in output_dict.items():
	
	output_dict[k] = pd.concat(v)

In [23]:
sample_file_df = pd.concat(output_dict.values())
sample_file_df.to_csv('uw_y4_sample_file_hifi_qc_aggregate.csv',index=False)
# 